In [ ]:
import mysql.connector
import pandas as pd 
import numpy as np 
from config import HOST, DATABASE, USERNAME, PASSWORD

In [ ]:
# connecting to grocery db

try:
    connection = mysql.connector.connect(
        host=HOST,
        database=DATABASE,
        user=USERNAME,
        password=PASSWORD
    )

    if connection.is_connected():
        print("Connected to MySQL database")

        # Your database operations go here

except mysql.connector.Error as err:
    print(f"Error: {err}")

# finally:
    # Close the connection in the finally block to ensure it's always closed
    #if 'connection' in locals() and connection.is_connected():
    #    connection.close()
    #    print("Connection closed")

cursor = connection.cursor() 

Connected to MySQL database


In [ ]:
items = ("SELECT * FROM ITEMS")
cursor.execute(items)

In [ ]:
col_info = cursor.description
col_types = [col[1] for col in col_info] 

In [ ]:
print(col_types)
# these are the data types - each number corresponds to a specific data type in mysql. 
'''
    2: MySQL type TINYINT
    253: MySQL type VAR_STRING (VARCHAR)
    246: MySQL type NEWDECIMAL
'''




[2, 253, 253, 246, 246, 246, 253, 246]


In [ ]:
store_data =[]
for i in cursor: 
    store_data.append(i)

col_names = [i[0] for i in cursor.description]

df = pd.DataFrame(store_data, columns = col_names)

conditions = [2,253,256]
choices = []
df


,trip_id,product_name,quantity_unit,quantity,price_per_unit,total_price,sku,weight_oz
0,1,APPLES COSMIC CRISP,LBS,2.79,2.50,6.92,84747300763,None
1,1,BANANAS,LBS,2.53,0.65,1.64,64312604011,None
2,1,KROGER SUPER,PC,1.00,2.50,2.50,1111002660,None
3,1,LACINATO KALE,PC,1.00,1.99,1.99,81490501036,None
4,1,NECTARINES ORGANIC,LBS,1.60,3.49,5.55,74183993035,None
...,...,...,...,...,...,...,...,...
362,43,CUCUMBER,PC,1.00,1.49,1.49,71106994062,None
363,43,HONEY TURKEY SLICED DELI,PC,1.00,8.57,8.57,20593890000,96
364,43,KROGER AMERICAN CHEESE DELI,PC,1.00,7.03,7.03,20635090000,None
365,43,SPINDRIFT 8 PK GRAPEFRUIT,PC,1.00,6.79,6.79,85657900232,96


In [ ]:
isinstance(df['product_name'], object) 

True

In [ ]:
# okay I have successfully queried data from the mysql db and stored it in a pandas dataframe
#let's formalize this process by creating a function

In [ ]:
#making sure I know how to parameterize a query...while avoiding an sql injection
def table_grabber(table):
    
    query = "SELECT * from {}"
    query_ready = query.format(table)

    cursor.execute(query_ready)

    data =[] # empty list to dump queried data into
    for x in cursor: 
        data.append(x)
    
    col_names = [i[0] for i in cursor.description]

    df = pd.DataFrame(data, columns = col_names)

    #coercing columns read in as 'object' that need to be numeric 
    #num_cols = df.select_dtypes(include = ['object']).columns
    #df[num_cols] = df[num_cols].apply(pd.to_numeric, errors = 'coerce')

    return df


In [ ]:
# my lovely tables 
stores = table_grabber("store")
trips = table_grabber("trips")
items = table_grabber("items")

InternalError: Unread result found

In [ ]:
#that didn't work, now it casting any and all objects as numerics. 

KeyError: 'trip_id'

In [ ]:
#what is average cost per grocery trip ? 

trips['total_cost'].mean() 

#$ 40.20 -- this aligns with my intuition. As a child I would accompany my mom to buy groceries. Her budget was usually $40. 

40.189756097560974

In [ ]:
#how about average monthly grocery expenses? 

# I could do this easily in R with %>% and group_by(), but I need to refresh my pandas skills....

#first I need to make sure the date column in trips is a datetime object: 

trips.dtypes

id             int64
store_id       int64
date          object
total_cost    object
dtype: object

In [ ]:
trips['date'] = pd.to_datetime(trips['date'])

In [ ]:
trips.groupby(trips['date'].dt.strftime('%B'))['total_cost'].sum()

#dt.strftime is the accessor in pandas to grab a part of the datetime object.'%B' is the full month name
# here are some other options 

'''
    %Y: Year with century as a decimal number (e.g., 2022).
    %m: Month as a zero-padded decimal number (01, 02, ..., 12).
    %d: Day of the month as a zero-padded decimal number (01, 02, ..., 31).
    %H: Hour (00, 01, ..., 23).
    %M: Minute (00, 01, ..., 59).
    %S: Second (00, 01, ..., 59).
'''

# for example, the average per year: 

trips.groupby(trips['date'].dt.strftime('%Y'))['total_cost'].mean()

date
2023    39.411429
2024    41.866154
Name: total_cost, dtype: object

In [ ]:
# group by week: 
trips.groupby(trips['date'].dt.strftime('%U'))['total_cost'].sum()

date
00    119.82
02     37.10
03    144.12
04     74.42
05    127.83
06     40.97
36     18.65
37     87.11
38    157.14
39     79.44
40    115.12
41     15.47
42     35.53
43     46.03
44      9.97
45    105.22
46     20.60
47    118.14
48     85.66
49     14.38
50     96.07
51     98.99
Name: total_cost, dtype: object

In [ ]:
# How many trips per week on average do I go to the grocery store? 
trips.groupby(trips['date'].dt.strftime('%B'))['id'].count()

date
December     6
February     6
January      7
November     8
October      7
September    7
Name: id, dtype: int64

In [ ]:
#summary statistics of trips table: 
trips.describe()


,id,store_id,date
count,41.000000,41.000000,41
mean,21.926829,4.560976,2023-11-27 15:48:17.560975616
min,1.000000,1.000000,2023-09-07 00:00:00
25%,11.000000,2.000000,2023-10-15 00:00:00
50%,21.000000,4.000000,2023-11-27 00:00:00
75%,33.000000,8.000000,2024-01-18 00:00:00
max,43.000000,10.000000,2024-02-16 00:00:00
std,12.861552,3.162663,NaN


In [ ]:
print(trips.dtypes)

id                     int64
store_id               int64
date          datetime64[ns]
total_cost            object
dtype: object


In [ ]:
# i've discovered that upon pulling the data for the trips table, the total_cost column was not identified as numeric, so it was being exluded from trips.describe()
#fix:
trips['total_cost'] = pd.to_numeric(trips['total_cost'], errors = 'coerce')

In [ ]:
print(items.dtypes)

trip_id            int64
product_name      object
quantity_unit     object
quantity          object
price_per_unit    object
total_price       object
sku               object
weight_oz         object
dtype: object


In [ ]:
#how many days/months does this dataset cover ? 
time_span = trips['date'].max() - trips['date'].min()

# duration in months - a bit over 5 months of shopping data 
time_span.days / 30



5.4

In [ ]:
# let's do something fun. I want to know the frequency of trips per grocery store. 
# to do this, I will need to join trips with store.

trip_freq_per_store = "SELECT s.name, count(t.date) FROM store as s JOIN trips as t ON s.id = t.store_id GROUP BY name"

cursor.execute(trip_freq_per_store)


In [ ]:
#generalizing table grabber to accept any query, also using cursor.fetchall() command

def table_grabber2(query):

    cursor.execute(query)

    query_data = cursor.fetchall()
    
    col_names = [i[0] for i in cursor.description]

    df = pd.DataFrame(data, columns = col_names)

    return df

In [ ]:
table_grabber2(trip_freq_per_store)

InternalError: Unread result found

In [1]:
#scratch 
datatypes = {2: int, 253: str, 246: float, 10: dt.datetime }

    relevant_data = [(t[0],t[1]) for t in cursor.description]
    relevant_dict = {key:value for key,value in relevant_data}

    for key, value in relevant_dict.items(): 
        if value in datatypes: 
            relevant_dict[key] = datatypes[value]


    df = pd.DataFrame(selected_table, columns = [i[0] for i in cursor.description])

    df.astype(dtype = relevant_dict)

    print(df.dtypes)

In [15]:
# libraries
import mysql.connector
import pandas as pd 
import numpy as np 
import datetime as dt

#import credentials for connecting to db from config.py file
from config import HOST, DATABASE, USERNAME, PASSWORD

#establishing connection to grocery db

def open_connection(): 
    try:
        connection = mysql.connector.connect(
            host=HOST,
            database=DATABASE,
            user=USERNAME,
            password=PASSWORD
        )

        if connection.is_connected():
            cursor = connection.cursor()
            print("Connected to GROCERY database.")

            return connection, cursor 

    except mysql.connector.Error as err:
        print(f"Error: {err}")


In [18]:
connection, cursor = open_connection()

Connected to GROCERY database.


In [19]:
print(cursor)

CMySQLCursor: (Nothing executed yet)


In [20]:
def import_tables(cursor):
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()
    return tables


In [55]:
tables = import_tables(cursor)



In [56]:
all_table_names = [table[0] for table in tables]

In [57]:
print(all_table_names)

['gas', 'items', 'pb_compare', 'peanut_butter_view', 'store', 'trips']


In [62]:
#create a dictionary of tables based on all_table_names

def table_generator(all_tables, connection): 

    table_dictionary = {}

    for table_name in all_tables:
        query = "SELECT * FROM " + table_name
        table = pd.read_sql(query, connection, parse_dates = ['date'], coerce_float= True)
        
        table_dictionary[table_name] = table

    return table_dictionary
    

In [63]:
grocery_tables = table_generator(all_table_names, connection)

C:\Users\chanks\AppData\Local\Temp\ipykernel_14040\2782467496.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table = pd.read_sql(query, connection, parse_dates = ['date'], coerce_float= True)
C:\Users\chanks\AppData\Local\Temp\ipykernel_14040\2782467496.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table = pd.read_sql(query, connection, parse_dates = ['date'], coerce_float= True)


In [67]:
grocery_tables['trips'].dtypes

id                     int64
store_id               int64
date          datetime64[ns]
total_cost           float64
dtype: object